![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite.png)
# <span style="color: #7b73e2;">WherobotsAI Raster Inference - Predicting Bounding Boxes and Segments of Airplanes from an LLM-based Text Query</span>


#### Learn how to perform text-based object detection on aerial imagery using WherobotsAI's Raster Inference with <span style="color: #7b73e2;">**Segment Anything Model 2 (SAM2)**</span> and <span style="color: #7b73e2;">**Google Deepmind's Open Vocabulary Object Detection (OWLv2)**</span>.
**Note:** Running Raster Inference notebooks requires a GPU-Optimized runtime and a Professional or Enterprise Edition Organization. If you haven't already done so, file a [**compute request**](https://docs.wherobots.com/latest/develop/runtimes/#compute-requests) to obtain a GPU-Optimized runtime.

#### Follow along as we use the simple term, \"airplanes\", to identify, segment, and draw bounding boxes around <span style="color: #7b73e2;">**commercial aircraft**</span> in satellite imagery of Miami airport.

## <span style="color: #7b73e2;" >Start WherobotsDB</span>

In [ ]:
from sedona.spark import SedonaContext
from sedona.raster_utils.SedonaUtils import SedonaUtils
from sedona.maps.SedonaKepler import SedonaKepler
from pyspark.sql.functions import expr

config = (
    SedonaContext.builder()
    .getOrCreate()
)

sedona = SedonaContext.create(config)

## <span style="color: #7b73e2;">Load Aerial Imagery Efficiently</span>

In this step, we'll load the aerial imagery so we can run <span style="color: #7b73e2;">**inference**</span> in a later step.

The GeoTIFF image is large, so we'll split it into tiles and load those tiles as <span style="color: #7b73e2;">**out-of-database**</span> or <span style="color: #7b73e2;">**\"out-db\" rasters**</span> in **WherobotsDB**.

In [ ]:
url = "s3://wherobots-examples/data/naip/miami-airport.tiff"
tile_size = 256
df = sedona.read.format("raster").option("tileWidth", tile_size).option("tileHeight", tile_size).load(url)
df.createOrReplaceTempView("df")
df.show()

## <span style="color: #7b73e2;">Viewing the Model's Imagery Inputs</span>

We can see the footprints of the tiled images with the `SedonaKepler.create_map()` integration. Using `SedonaUtils.display_image()` we can view the images as well.

<span style="color: #7b73e2;;"> **Tip:** </span>  Save the map to a html file using `kepler_map.save_to_html()`

In [ ]:
kepler_map = SedonaKepler.create_map()
df = df.withColumn('footprint', expr("ST_TRANSFORM(RS_CONVEXHULL(rast),'EPSG:4326')"))
SedonaKepler.add_df(kepler_map, df=df, name="Image Footprints")

kepler_map

In [ ]:
htmlDf = sedona.sql(f"""SELECT RS_AsImage(rast, 250) as FROM df limit 5""")
SedonaUtils.display_image(htmlDf)

## <span style="color: #7b73e2;">Run Inference and Visualize Results</span>

To run inference, specify the model to use with `model id`. Five models are pre-loaded and made available in **Wherobots Cloud** to Professional and Enterprise customers. You can also load your own models, learn more about that process [here](https://docs.wherobots.com/latest/tutorials/wherobotsai/wherobots-inference/raster-inference-overview/#bring-your-own-model-guide).

Inference can be run using **Wherobots' Spatial SQL functions**, in this case: `RS_Text_to_Segments()`.
            
Here, we generate predictions for all images in the Region of Interest (ROI). In the output, a label value of 1 signifies a positive prediction corresponding to the input text prompt.

Then, we'll filter and print some of the results to see how our positive detection results look.

In [ ]:
model_id = "sam2"
prompt = "airplanes"
threshold = 0.5

preds = sedona.sql(
    f"""SELECT rast, RS_TEXT_TO_SEGMENTS('{model_id}', rast, '{prompt}', {threshold}) AS preds from df"""
)
preds.cache().count()
preds.createOrReplaceTempView("preds")

## <span style="color: #7b73e2;">Prepare Results</span>

Before plotting our predictions, we need to transform our results.

We'll need to transform our table so that each raster scene _only_ corresponds to a single predicted bounding box instead of every bounding box prediction.

<span style="color: #7b73e2;">**Bounding boxes (or Bboxes)**</span> are essentially boundaries drawn around an object of interest. 

To do this, combine the list columns containing our prediction results (`max_confidence_bboxes`, `max_confidence_scores`, and `max_confidence_labels`) with `arrays_zip`.  Then, use `explode` to convert lists to rows.

To map the results with `SedonaKepler`, convert the `max_confidence_bboxes` column to a `GeometryType` column with `ST_GeomFromWKT`

In [ ]:
preds_filtered = sedona.sql(f"""
  SELECT *
  FROM preds
  WHERE
    size(preds.labels) > 0
    AND array_contains(preds.labels, 1)
    AND NOT array_contains(preds.segments_wkt, 'POLYGON EMPTY')
""")
preds_filtered.createOrReplaceTempView("preds_filtered")
preds_filtered.show()

In [ ]:
exploded = sedona.sql("""
SELECT
    rast,
    exploded_predictions.*
FROM
    preds_filtered
LATERAL VIEW explode(arrays_zip(preds.segments_wkt, preds.confidence_scores, preds.labels)) AS exploded_predictions
WHERE
    exploded_predictions.confidence_scores != 0.0
""")
exploded.cache().count()
exploded.createOrReplaceTempView("exploded")
exploded.show()

## <span style="color: #7b73e2;">Viewing Model Results: Airplane Segmentation Predictions</span>

Just like we visualized the footprints of the tiled images earlier, we can also view our prediction geometries! Highlight a prediction to view its confidence score.

In [ ]:
kepler_map = SedonaKepler.create_map()
SedonaKepler.add_df(kepler_map, df=exploded, name="Airplane Detections")

kepler_map

To view results on the underlying imagery used by the model, you can use the `show_detections` function. This function accepts a Dataframe containing an `outdb_raster` column as well as other arguments to control the plot result. Check out the full docs for the function by calling `show_detections?`

In [ ]:
from wherobots.inference.plot.detections import show_detections
show_detections?

In [ ]:
unpacked_preds_df = sedona.sql("SELECT rast, preds.* FROM preds_filtered")

In [ ]:
show_detections(
    unpacked_preds_df,
    confidence_threshold=0.7,
    plot_geoms=True,
    geometry_column="segments_wkt",
)

## <span style="color: #7b73e2;">Running Object Detection with a Text Prompt</span>

We can also get bounding box predictions instead of segments using `RS_Text_To_BBoxes`. BBoxes, or bounding boxes, are more useful when you are only concerned with counting and localizing objects rather than delineating exact shape and area with `RS_Text_To_Segments`.

The inference process is largely the same for `RS_Text_To_BBoxes` and `RS_Text_To_Segments`.

There are 2 key differences:
* Using the `owlv2` `model_id` instead of `sam2`.
* Changing our SQL queries to operate on the `bboxes_wkt` column instead of the `segments_wkt` column when working with prediction results.

In [ ]:
model_id = "owlv2"
prompt = "airplanes"
threshold = 0.5

preds = sedona.sql(
    f"""SELECT rast, RS_TEXT_TO_BBoxes('{model_id}', rast, '{prompt}', {threshold}) AS preds from df"""
)
preds.cache().count()
preds.createOrReplaceTempView("preds")

Just like before, we'll filter predictions by labels, remove empty predictions, and show the results in a browsable map and on top of the original imagery for comparison.

In [ ]:
preds_filtered = sedona.sql(f"""
  SELECT *
  FROM preds
  WHERE
    size(preds.labels) > 0
    AND array_contains(preds.labels, 1)
    AND NOT array_contains(preds.bboxes_wkt, 'POLYGON EMPTY')
""")
preds_filtered.createOrReplaceTempView("preds_filtered")
preds_filtered.show()

In [ ]:
exploded = sedona.sql("""
SELECT
    rast,
    exploded_predictions.*
FROM
    preds_filtered
LATERAL VIEW explode(arrays_zip(preds.bboxes_wkt, preds.confidence_scores, preds.labels)) AS exploded_predictions
WHERE
    exploded_predictions.confidence_scores != 0.0
""")
exploded.cache().count()
exploded.createOrReplaceTempView("exploded")
exploded.show()

In [ ]:
kepler_map = SedonaKepler.create_map()
SedonaKepler.add_df(kepler_map, df=exploded, name="Airplane Detections")

kepler_map

In [ ]:
unpacked_preds_df = sedona.sql("SELECT rast, preds.* FROM preds_filtered")

We see below that OWLv2 and SAM2 do remarkably well at identifying airplanes with little user effort! Previously, achieving similar results was a significant undertaking. An entire Machine Learning engineering team would have needed to build such a model from scratch.

In [ ]:
show_detections(
    unpacked_preds_df,
    confidence_threshold=0.5,
    plot_geoms=True,
    side_by_side=False,
    geometry_column="bboxes_wkt",
)

## <span style="color: #7b73e2;">Next Steps with Raster Inference</span>

With access to general-purpose, text-promptable models, what will you predict and georeference next?

Some ideas on next steps to try, include:

* Predicting different objects next to the airplanes in the image tiles above using new text prompts.
* Adjusting the confidence score threshold for `RS_Text_to_Segments` or `RS_Text_to_BBoxes` to see how SAM2 or OWLv2 respond.
* Loading a new imagery dataset with our [STAC Reader](https://docs.wherobots.com/latest-snapshot/references/wherobotsdb/vector-data/Stac/) and try to predict a different feature of interest, such as agriculture, buildings, or tree crowns.

We're excited to hear about what you're doing with SAM2 and OWLv2! 